In [4]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *   # module Tracker return coordinates and id
import torch

model = YOLO('yolov8s.pt')

# Set the device to CPU
device = torch.device('cpu')

# Define the areas
area1 = [(312, 388), (289, 390), (474, 469), (497, 462)]
area2 = [(279, 392), (250, 397), (423, 477), (454, 469)]

# Define the mouse event function
def print_mouse_position(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        position = [x, y]
        print(position)

# Set up the OpenCV window and mouse callback
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', print_mouse_position)

# Open the video file
cap = cv2.VideoCapture("C:/Users/ASUS/Downloads/project/peoplecount1.mp4")

# Open and read the class file
with open("coco.txt", "r") as my_file:
    data = my_file.read()
    class_list = data.split("\n") 

# Initialize dictionaries and sets
people_entering = {}
entering = set()
people_exiting = {}
exiting = set()
tracker = Tracker()


while True:    
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (1020, 500))
    results = model.predict(frame)
    a = results[0].boxes.data.cpu().numpy()  # Move tensor to CPU and convert to NumPy array
    px = pd.DataFrame(a).astype("float")
    #print(px)
    person_coordinates = [] # set coordinate of person 
    for index, person_data in px.iterrows():
        x1, y1, x2, y2, class_index = map(int, [person_data[0], person_data[1], person_data[2], person_data[3], person_data[4]])
        class_name = class_list[class_index]
        if 'person' in class_name:
            person_coordinates.append([x1, y1, x2, y2])  
           
    bbox_id = tracker.update(person_coordinates)  # module Tracker return coordinates and id
    
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
       # people entering
        if cv2.pointPolygonTest(np.array(area2, np.int32), ((x4, y4)), False) >= 0: # check if coordinates of people inside regions of interest 2
            people_entering[id] = (x4, y4)
            cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2)
            
        if id in people_entering:
            if cv2.pointPolygonTest(np.array(area1, np.int32), ((x4, y4)), False) >= 0: # check if coordinates of people inside regions of interest 1
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                cv2.circle(frame, (x4, y4), 4, (255, 0, 255), -1)
                cv2.putText(frame, str(c), (x3, y3-10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                cv2.putText(frame, str(id), (x3+65, y3-10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 255), 1)
                entering.add(id)
        
        # people exiting
        if cv2.pointPolygonTest(np.array(area1, np.int32), ((x4, y4)), False) >= 0: # check if coordinates of people inside regions of interest 1
            people_exiting[id] = (x4, y4)
            cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
        if id in people_exiting:
            if cv2.pointPolygonTest(np.array(area2, np.int32), ((x4, y4)), False) >= 0: # check if coordinates of people inside regions of interest 2
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2)
                cv2.circle(frame, (x4, y4), 4, (255, 0, 255), -1)
                cv2.putText(frame, str(c), (x3, y3-10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                cv2.putText(frame, str(id), (x3+55, y3-10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 255), 1)
                exiting.add(id)
            
            
    cv2.polylines(frame, [np.array(area1, np.int32)], True, (255, 0, 0), 2) # draw Regions of Interest 1
    cv2.putText(frame, str('1'), (504, 471), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)

    cv2.polylines(frame, [np.array(area2, np.int32)], True, (255, 0, 0), 2) # draw Regions of Interest 2
    cv2.putText(frame, str('2'), (466, 485), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
    
    i = len(entering)
    o = len(exiting)
    cv2.putText(frame, 'Number of entering people= '+str(i), (20, 44), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, 'Number of exiting people= '+str(o), (20, 82), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
    

    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:
        break

cap.release()
cv2.destroyAllWindows()


0: 320x640 1 car, 47.3ms
Speed: 15.8ms preprocess, 47.3ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 23.4ms
Speed: 3.1ms preprocess, 23.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 20.6ms
Speed: 0.0ms preprocess, 20.6ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 12.2ms
Speed: 6.9ms preprocess, 12.2ms inference, 2.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 car, 17.6ms
Speed: 0.0ms preprocess, 17.6ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 car, 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 1 car, 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 car, 13.9ms
Speed: 1.0ms preprocess, 13.9ms inference, 1.0ms postprocess per image 